Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import time
localtime = time.asctime( time.localtime(time.time()) )
print (localtime)

Thu Jun 30 17:04:07 2016


First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (5000, 28, 28) (5000,)
Test set (5000, 28, 28) (5000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 784) (20000, 10)
Validation set (5000, 784) (5000, 10)
Test set (5000, 784) (5000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [14]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [7]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [15]:
num_steps = 4001

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 400 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.2f%%' % accuracy(
        predictions, train_labels[:train_subset]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.2f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print ('Test accuracy: %.2f%%' % accuracy(test_prediction.eval(), test_labels)) 

Initialized
Loss at step 0: 14.480868
Training accuracy: 18.35%
Validation accuracy: 26.36%
Loss at step 400: 1.084673
Training accuracy: 79.11%
Validation accuracy: 76.12%
Loss at step 800: 0.799964
Training accuracy: 81.85%
Validation accuracy: 76.74%
Loss at step 1200: 0.667826
Training accuracy: 83.59%
Validation accuracy: 77.46%
Loss at step 1600: 0.589473
Training accuracy: 84.94%
Validation accuracy: 77.30%
Loss at step 2000: 0.536895
Training accuracy: 86.29%
Validation accuracy: 77.24%
Loss at step 2400: 0.498685
Training accuracy: 87.09%
Validation accuracy: 77.16%
Loss at step 2800: 0.469332
Training accuracy: 87.78%
Validation accuracy: 77.30%
Loss at step 3200: 0.445866
Training accuracy: 88.15%
Validation accuracy: 77.46%
Loss at step 3600: 0.426573
Training accuracy: 88.63%
Validation accuracy: 77.52%
Loss at step 4000: 0.410366
Training accuracy: 89.09%
Validation accuracy: 77.44%
Test accuracy: 84.40%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `sesion.run()`.

In [20]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [25]:
num_steps = 3001
loss_list = []
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    # Generate a minibatch.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #batch_data = train_dataset[offset:(offset + batch_size), :]
    #batch_labels = train_labels[offset:(offset + batch_size), :]
    
    
    #Using permutation to randomnilized the data
    permutation = np.random.permutation(train_labels.shape[0])
    idx = permutation[:batch_size]
    #Generate a minibatch
    batch_data = train_dataset[idx, :]
    batch_labels = train_labels[idx,:]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    loss_list.append(l)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  

Initialized
Minibatch loss at step 0: 15.716947
Minibatch accuracy: 8.6%
Validation accuracy: 8.2%
Minibatch loss at step 500: 2.239144
Minibatch accuracy: 75.0%
Validation accuracy: 70.3%
Minibatch loss at step 1000: 2.010792
Minibatch accuracy: 71.1%
Validation accuracy: 73.3%
Minibatch loss at step 1500: 1.742933
Minibatch accuracy: 75.8%
Validation accuracy: 73.9%
Minibatch loss at step 2000: 1.015141
Minibatch accuracy: 82.0%
Validation accuracy: 74.8%
Minibatch loss at step 2500: 1.927985
Minibatch accuracy: 74.2%
Validation accuracy: 75.2%
Minibatch loss at step 3000: 1.310254
Minibatch accuracy: 82.0%
Validation accuracy: 75.8%
Test accuracy: 82.7%


In [24]:
import matplotlib.pyplot as plt
plt.plot(loss_list)
plt.show()

---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units (nn.relu()) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [26]:
batch_size = 128
n_hidden = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  tf_hidden_dataset = tf.placeholder(tf.float32, shape = (batch_size, n_hidden))
    
  

  # Variables.
  weights_0 = tf.Variable(
    tf.truncated_normal([image_size * image_size, n_hidden]))
  biases_0 = tf.Variable(tf.zeros([n_hidden]))
    
  weights_1 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  biases_1 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden_level = tf.matmul(tf_train_dataset, weights_0) + biases_0
  tf_hidden_dataset = tf.nn.relu(hidden_level)
  logits = tf.matmul(tf_hidden_dataset, weights_1) + biases_1
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  

  def multilayer_perceptron(x, w_0, b_0,w_1, b_1):
        l1 = tf.nn.relu(tf.matmul(x, w_0) + b_0)
        pred = tf.matmul(l1, w_1) + b_1
        return pred
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(multilayer_perceptron(tf_valid_dataset, weights_0, biases_0, weights_1, biases_1))
  test_prediction = tf.nn.softmax(multilayer_perceptron(tf_test_dataset, weights_0, biases_0, weights_1, biases_1))
  #valid_prediction = tf.nn.softmax(
  #  tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weights_0) + biases_0), weights_1) + biases_1)
  #test_prediction = tf.nn.softmax(
  #  tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_0) + biases_0), weights_1) + biases_1))
  print (r'Run the following model!')

Run the following model!


Let's run it:

In [38]:
num_steps = 1001
rows = train_labels.shape[0]
loss_list = []
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    permutation = np.random.permutation(train_labels.shape[0])
    idx = permutation[:batch_size]
    #Generate a minibatch
    batch_data = train_dataset[idx, :]
    batch_labels = train_labels[idx,:]
    # Generate a minibatch.
    #batch_data = train_dataset[offset:(offset + batch_size), :]
    #batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    loss_list.append(l)
    if (step % 200 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.2f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.2f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 307.685669
Minibatch accuracy: 7.03%
Validation accuracy: 35.78%
Minibatch loss at step 200: 23.557085
Minibatch accuracy: 79.69%
Validation accuracy: 79.28%
Minibatch loss at step 400: 4.324033
Minibatch accuracy: 89.06%
Validation accuracy: 81.52%
Minibatch loss at step 600: 3.972778
Minibatch accuracy: 92.19%
Validation accuracy: 81.28%
Minibatch loss at step 800: 11.069275
Minibatch accuracy: 92.19%
Validation accuracy: 79.68%
Minibatch loss at step 1000: 2.973181
Minibatch accuracy: 92.97%
Validation accuracy: 81.66%
Test accuracy: 88.66%


In [39]:
import matplotlib.pyplot as plt
plt.plot(loss_list)
plt.show()

In [16]:
batch_size = 128
n_hidden = [32, 32]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  #tf_hidden_dataset = tf.placeholder(tf.float32, shape = (batch_size, n_hidden))
    
  

  # Variables.
  #weights_0 = tf.Variable(
  #  tf.truncated_normal([image_size * image_size, n_hidden]))
  weights = [tf.truncated_normal([image_size * image_size, n_hidden[0]]),
             tf.truncated_normal([n_hidden[0], n_hidden[1]]),
             tf.truncated_normal([n_hidden[1], num_labels])]
  #biases_0 = tf.Variable(tf.zeros([n_hidden]))
  #biases_0 = tf.Variable(tf.truncated_normal([n_hidden]))
  biases = [tf.Variable(tf.zeros([n_hidden[0]])),
            tf.Variable(tf.zeros([n_hidden[1]])), 
            tf.Variable(tf.zeros([num_labels]))]  
  #weights_1 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  #biases_1 = tf.Variable(tf.zeros([num_labels]))
  #biases_1 = tf.Variable(tf.truncated_normal([num_labels]))
  
  # Training computation.
  def multilayer_perceptron2(x, w, b):
        h1 = tf.nn.relu(tf.matmul(x, w[0]) + b[0])
        h2 = tf.nn.relu(tf.matmul(h1, w[1]) + b[1])
        pred = tf.matmul(h2, w[2]) + b[2]
        return pred
            
  #hidden_level_1 = tf.matmul(tf_train_dataset, weights_0) + biases_0
  #tf_hidden_dataset = tf.nn.softmax(hidden_level)
  #logits = tf.matmul(tf_hidden_dataset, weights_1) + biases_1,
  logits = multilayer_perceptron2(tf_train_dataset, weights, biases)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(1e-20).minimize(loss)
  
  
  
  
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(multilayer_perceptron2(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(multilayer_perceptron2(tf_test_dataset, weights,biases))
  #valid_prediction = tf.nn.softmax(
  #  tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weights_0) + biases_0), weights_1) + biases_1)
  #test_prediction = tf.nn.softmax(
  #  tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_0) + biases_0), weights_1) + biases_1))
  print (r'Run the following model!')

Run the following model!


In [17]:
num_steps =1001
rows = train_labels.shape[0]
loss_list = []

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    permutation = np.random.permutation(train_labels.shape[0])
    idx = permutation[:batch_size]
    #Generate a minibatch
    batch_data = train_dataset[idx, :]
    batch_labels = train_labels[idx,:]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    loss_list.append(l)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  print (type(train_prediction))
  print (type(test_prediction))
  test_pred = test_prediction.eval()
  print (type(test_pred))
  print (test_pred.shape)
  out = np.argmax(test_pred, 1)
  print (out.shape)
  #out = pd.DataFrame(out)
  #out.to_csv('out.csv', header = ['Label'], index_label = range(1,test_pred.shape[0]+1))
  #print ('done!')

Initialized
Minibatch loss at step 0: 250.802521
Minibatch accuracy: 8.6%
Validation accuracy: 10.8%
Minibatch loss at step 100: 225.749756
Minibatch accuracy: 10.9%
Validation accuracy: 10.8%
Minibatch loss at step 200: 288.128662
Minibatch accuracy: 5.5%
Validation accuracy: 5.6%
Minibatch loss at step 300: 142.956726
Minibatch accuracy: 16.4%
Validation accuracy: 11.3%
Minibatch loss at step 400: 151.995422
Minibatch accuracy: 16.4%
Validation accuracy: 10.4%
Minibatch loss at step 500: 218.672211
Minibatch accuracy: 11.7%
Validation accuracy: 13.8%
Minibatch loss at step 600: 168.934799
Minibatch accuracy: 8.6%
Validation accuracy: 10.3%
Minibatch loss at step 700: 168.826523
Minibatch accuracy: 17.2%
Validation accuracy: 10.8%
Minibatch loss at step 800: 208.683426
Minibatch accuracy: 12.5%
Validation accuracy: 11.0%
Minibatch loss at step 900: 155.337082
Minibatch accuracy: 16.4%
Validation accuracy: 8.0%
Minibatch loss at step 1000: 284.282074
Minibatch accuracy: 9.4%
Validation

In [19]:
import matplotlib.pyplot as plt
plt.plot(loss_list)
plt.show()